# ASSET BACKED SECURITIES - AUTO FINANCE LOANS

SEC data ingested from Edgar Online

In [ ]:
import re
import time
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, GRU
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
tf.__version__
%matplotlib inline

import seaborn as sns
import itertools
import operator
import collections
from scipy.io import mmread, mmwrite
from random import randint
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing as pp
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.decomposition import PCA, KernelPCA
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
import scipy.stats as stats
from sklearn import tree
from sklearn.feature_selection import f_regression
from sklearn.gaussian_process import GaussianProcess
from sklearn.metrics import roc_auc_score
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler





# Model Name

In [ ]:
name = "ABS_Keras-GRU254GRU128D32D2-adam{}".format(int(time.time()))
#name_noTime = "ABS_Keras-LSTM128LSTM128D32D2-adam"

# Choose default parameters

In [ ]:
#parameters to choose

default_days = 0
nrows = 1000000
train_split = 0.80
model_type = keras
path = 'c:/users/ugy1/abs/'
rows_for_prediction = 1000
cutOffForOriginationDate='2012-01-01'
manualOversamplingFactor=0.0
epochs=1

In [ ]:
# columns to be used
use_list=[
    'abs_loan.cik',
 #'abs_loan.accessionnumber',
 #'abs_loan.assettypenumber',
 'abs_loan.assetnumber',
 'abs_loan.reportingperiodbeginningdate',
 'abs_loan.reportingperiodendingdate',
 'abs_loan.originatorname',
 'abs_loan.originationdate',
 'abs_loan.originalloanamount',
 'abs_loan.originalloanterm',
 'abs_loan.loanmaturitydate',
 'abs_loan.originalinterestratepercentage',
 #'interestcalculationtypecode',
 #'originalinterestratetypecode',
 #'originalinterestonlytermnumber',
 'abs_loan.originalfirstpaymentdate',
 'abs_loan.underwritingindicator',
 'abs_loan.graceperiodnumber',
 #'paymenttypecode',
 'abs_loan.subvented',
 #'vehiclemanufacturername',
 #'vehiclemodelname',
 'abs_loan.vehiclenewusedcode',
 #'vehiclemodelyear',
 'abs_loan.vehicletypecode',
 #'vehiclevalueamount',
 #'vehiclevaluesourcecode',
 'abs_loan.obligorcreditscoretype',
 'abs_loan.obligorcreditscore',
 'abs_loan.obligorincomeverificationlevelcode',
 'abs_loan.obligoremploymentverificationcode',
 'abs_loan.coobligorindicator',
 'abs_loan.paymenttoincomepercentage',
 #'obligorgeographiclocation',
 'abs_loan.assetaddedindicator',
 #'remainingtermtomaturitynumber',
 'abs_loan.reportingperiodmodificationindicator',
 'abs_loan.servicingadvancemethodcode',
 'abs_loan.reportingperiodbeginningloanbalanceamount',
 'abs_loan.nextreportingperiodpaymentamountdue',
 'abs_loan.reportingperiodinterestratepercentage',
 'abs_loan.nextinterestratepercentage',
 'abs_loan.servicingfeepercentage',
 'abs_loan.servicingflatfeeamount',
 'abs_loan.otherservicerfeeretainedbyservicer',
 'abs_loan.otherassesseduncollectedservicerfeeamount',
 'abs_loan.scheduledinterestamount',
 'abs_loan.scheduledprincipalamount',
 'abs_loan.otherprincipaladjustmentamount',
 'abs_loan.reportingperiodactualendbalanceamount',
 'abs_loan.reportingperiodscheduledpaymentamount',
 'abs_loan.totalactualamountpaid',
 'abs_loan.actualinterestcollectedamount',
 'abs_loan.actualprincipalcollectedamount',
 'abs_loan.actualothercollectedamount',
 'abs_loan.serviceradvancedamount',
 'abs_loan.interestpaidthroughdate',
 'abs_loan.zerobalanceeffectivedate',
 'abs_loan.zerobalancecode',
 'abs_loan.currentdelinquencystatus',
 #'primaryloanservicername',
 #'mostrecentservicingtransferreceiveddate',
 #'assetsubjectdemandindicator',
 #'assetsubjectdemandstatuscode',
 #'repurchaseamount',
 #'demandresolutiondate',
 #'repurchasername',
 #'repurchasereplacementreasoncode',
 #'chargedoffprincipalamount',
 #'recoveredamount',
 #'modificationtypecode',
 #'paymentextendednumber',
 #'repossessedindicator',
 #'repossessedproceedsamount'
]

In [ ]:
#df.drop(drop_list,axis=1, inplace=True)

# GPU Options

In [ ]:
#set gpu options to adjust gpu usage
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
# sess=tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# Load data

In [ ]:
#load data from tab delimited file
df=pd.read_csv(path+'datasets/abs_loan.csv', 
               usecols=use_list, 
               sep='\t',
               #compression=bz2,
               nrows=nrows,
               low_memory=False, 
              #index_col='abs_loan.reportingperiodendingdate', 
               parse_dates=True
              )

#remove 'abs_loan.' string from columns because 'dot' is a command in pandas
df.columns=df.columns.str.replace('abs_loan.','')

#display data shape
print(df.shape)
#view first five rows of all columns
#df.head(n=5)

In [ ]:
#df.cik.unique()

In [ ]:
#df.assetnumber.unique()

In [ ]:
#df.loc[:,~df.columns.str.contains('^Unnamed')]

In [ ]:
#df.duplicated(subset=['cik','assetnumber']).sum()

In [ ]:
#df.duplicated(subset=['cik','assetnumber','reportingperiodendingdate']).sum()

In [ ]:
df.duplicated(subset=['cik','assetnumber','reportingperiodendingdate','originationdate']).sum()

In [ ]:
#df.loc[df.duplicated(subset=['cik','assetnumber','reportingperiodendingdate']),:]

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(subset=['cik','assetnumber','reportingperiodendingdate','originationdate'],keep='last', inplace=True)

In [ ]:
#df.columns.tolist()
df.shape

In [ ]:
#df.info(memory_usage='deep')

In [ ]:
#df.memory_usage(deep=True).sum()

In [ ]:
#df.dtypes

In [ ]:
#df.head()
#df=df.sort_values(by='originationdate', ascending=True)

# Function to Draw Bar chart

In [ ]:
#function to draw a bar chart 
def draw(data, title, ylabel, sort = False, n=-1):
    labels = pd.value_counts(data).head().index.tolist()
    count_classes = pd.value_counts(data, sort).head()
    count_classes.plot(kind = 'bar', rot=0)
    plt.xticks(range(len(labels)), labels)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.show()
    print('Top ',n,' counts: ','\n',pd.value_counts(data, sort).head(n))

# Reporting Period

In [ ]:
draw(df.reportingperiodendingdate, title='reporting period endingdate', ylabel='Number of Loans', sort = True, n=100)

In [ ]:
# restrict data to a reporting period
#df=df[df.index=='2017-11-30']

Mean current deliquency days for the reporting period

In [ ]:
df.groupby(df.reportingperiodendingdate).currentdelinquencystatus.mean()

Mean ending balance for the reporting period

In [ ]:
df.groupby(df.reportingperiodendingdate).reportingperiodactualendbalanceamount.mean()

# Create Features for Reporting period

In [ ]:
# create feature columns for categories within originator name 
def map_reportingperiodendingdate(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.reportingperiodendingdate).index.tolist()

for code in codes:
        df['reportingperiodending_{}'.format(code)] = list(map(map_reportingperiodendingdate, df.reportingperiodendingdate))
        print('Added Column for :   reportingperiodending_'+code)
print('New Dataframe shape : ', df.shape)

# Number of Days Loans Outstanding

In [ ]:
#calculate outstanding days
df['outstandingdays'] = pd.to_datetime(df['reportingperiodendingdate'])-pd.to_datetime(df['originationdate'])
#draw a chart for visual look and analysis
draw(df.outstandingdays, title='Outstanding Days', ylabel='Number of Loans', sort = True, n=5)
#remove days string for machine learning
df['outstandingdays']=df.outstandingdays.map(lambda x: np.nan if pd.isnull(x) else x.days)
#display last five rows
#df.outstandingdays.tail()

In [ ]:
# retaining only loans with non-negative outstanding days
u=pd.value_counts(df.outstandingdays<0)
if len(u)<2:
    print('No Loans Below zero outstanding days')
else: 
    print('Number of Loans with negative outstanding days :', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with only non-negative outstanding days')
    df=df[df.outstandingdays>0]   

In [ ]:
# get statistics for outstanding days
df.outstandingdays.describe()

Mean outstanding days for the reporting period

In [ ]:
df.groupby(df.reportingperiodendingdate).outstandingdays.mean()

# Days Left until Maturity

In [ ]:
#calculate days left until maturity
df['daystomaturity'] = pd.to_datetime(df['loanmaturitydate'])-pd.to_datetime(df['reportingperiodendingdate'])
draw(df.daystomaturity, title='Days Left Until Maturity', ylabel='Number of Loans', sort=True, n=5)

In [ ]:
# remove days from the data for machine learning
df['daystomaturity']=df.daystomaturity.map(lambda x: np.nan if pd.isnull(x) else x.days)
# print last five rows
df.daystomaturity.tail()

In [ ]:
#retaining only loans with non-negative days to maturity
u=pd.value_counts(df.daystomaturity<=0)
if len(u)<2:
    print('No Loans Below zero outstanding days')
else: 
    print('Number of Loans with negative outstanding days :', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with only non-negative outstanding days')
    df=df[df.daystomaturity>0]

In [ ]:
#number of rows and columns of the dataframe
df.shape

In [ ]:
# statistics for days to maturity
df.daystomaturity.describe()

Mean days to maturity for the reporting period

In [ ]:
df.groupby(df.reportingperiodendingdate).daystomaturity.mean()

In [ ]:
df.drop(['loanmaturitydate'], axis=1, inplace=True)

# Origination date

# Removing loans before origination cutoff date

In [ ]:
# remove loans with orgination dates set for cut off earlier
u=pd.value_counts(df.originationdate<=cutOffForOriginationDate)
if len(u)<2:
    print('No origination date rows fall before ',cutOffForOriginationDate )
else: 
    print('Number of rows before',cutOffForOriginationDate ,':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with origination dates after', cutOffForOriginationDate)
    df=df[df.originationdate>cutOffForOriginationDate]
#new number of rows and columns of the dataframe
df.shape

In [ ]:
#set origination date as index
#df.set_index(['originationdate'])
# draw origination date chart
draw(df.originationdate, title='Origination Date', ylabel='Number of Loans', sort=True, n=10)

In [ ]:
# statistics for origination date
df.originationdate.describe()

Top ten mean current deliquncy status by origination date

In [ ]:
df.groupby([df.originationdate, df.reportingperiodendingdate]).currentdelinquencystatus.mean().sort_values().tail(n=10).plot(kind='bar')

In [ ]:
df.groupby(df.originationdate).currentdelinquencystatus.mean().sort_values().tail(n=10).plot(kind='bar')

# Originators Name

In [ ]:
df.originatorname.unique()

In [ ]:
# draw charts for originators name
draw(df.originatorname, title='Originators Name', ylabel='Number of Loans', sort=True)

# Create Features for Originators

In [ ]:
df['originatorname']=df.originatorname.astype('category')

In [ ]:
# create feature columns for categories within originator name 
def map_originatorname(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.originatorname).index.tolist()

for code in codes:
        df['originator_{}'.format(code)] = list(map(map_originatorname, df.originatorname))
        print('Added Column for :   originator_'+code)
print('New Dataframe shape : ', df.shape)

In [ ]:
# drop originator name column
df.drop(['originatorname'],axis=1, inplace=True)
# new number of rows and columns
df.shape

# Original Loan Amount

In [ ]:
df.groupby(df.reportingperiodendingdate).originalloanamount.mean()


In [ ]:
# draw chart for original loan amount as filered in the next variable(original_loan_in_excess) 
original_loan_in_excess = 10000
draw(df.originalloanamount>original_loan_in_excess, title='Original Loan Amount', ylabel='Number of Loans', sort=True)

In [ ]:
#draw a chart to view distribution of original loan amount
bins = np.linspace(0, 80000, 1000)
plt.hist(df.originalloanamount, bins, label='Original Loan Amount')
plt.legend(loc='upper right')
plt.title("Original Amount Loaned")
plt.xlabel('Original Loan amount')
plt.ylabel('Number of Loans')
plt.show()

In [ ]:
# get statistics for original loan amount
df.originalloanamount.describe()

In [ ]:
# To remove rows with zero, non-negative or blank values for original loan amount

u=pd.value_counts(df.originalloanamount<=0)
if len(u)<2:
    print('No original loan amount blank or below zero')
else: 
    print('Number of original loan amount rows blank or below zero',':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with original loan amount more than zero')
    df=df[df.originalloanamount>0]
df.shape

# Original Loan Term

In [ ]:
df.groupby(df.reportingperiodendingdate).originalloanterm.mean()

In [ ]:
# draw chart for original loan term based on the input of variable below(original_loan_term_excess)
original_loan_term_excess = 48
draw(df.originalloanterm>original_loan_term_excess, title='Original Loan Terms', ylabel='Number of Loans',sort=True)

In [ ]:
#chart for the distribution of original loan term
bins = np.linspace(0, 84, 12)
plt.hist(df.originalloanterm, bins, label='Original Loan Term')
plt.legend(loc='upper right')
plt.title("Original Loan Term")
plt.xlabel('Number of Months')
#plt.ylabel('Number of Loans')
plt.show()

In [ ]:
#statistics for original loan term
df.originalloanterm.describe()

In [ ]:
# remove rows with zero, blank or negative original loan term

u=pd.value_counts(df.originalloanterm<=0)
if len(u)<2:
    print('No original loan term blank or below zero')
else: 
    print('Number of original loan term rows blank or below zero',':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with original loan term more than zero')
    df=df[df.originalloanterm>0]
# new number of rows and columns of the dataframe
df.shape

# Original interest rate percentage

In [ ]:
# draw a chart with a filter input as below (originalinterestratepercentage_excess)
originalinterestratepercentage_excess = 2.0
draw(df.originalinterestratepercentage<originalinterestratepercentage_excess, title='Original Interest Rate Percentage', ylabel='Number of Loans',sort=True)

In [ ]:
# get statistics for originalinterestratepercentage
#df.originalinterestratepercentage.describe()

In [ ]:
# drop rows with blank or negative original interest rate percentage and research on zero interest rate loans
u=pd.value_counts(df.originalinterestratepercentage<0)
if len(u)<2:
    print('No original interest rate blank or below zero')
else: 
    print('Number of original interest rate rows blank or below zero',':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with original interest rate with non-negative values')
    df=df[df.originalinterestratepercentage>=0]
df.shape

# No interest rate loans

In [ ]:
# view zero original interest rate percentage loans
u=pd.value_counts(df.originalinterestratepercentage==0)
if len(u)<2:
    print('No zero original interest rate percentage')
else: 
    print('Number of zero original interest rate rows',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with original interest rate with non-negative values')
    #df=df[df.originalinterestratepercentage>=0]
df.shape

# Underwriting Indicator

In [ ]:
df.underwritingindicator.unique()

In [ ]:
df['underwritingindicator']=df.underwritingindicator.astype('category')

In [ ]:
draw(df.underwritingindicator, title='Underwriting Indicator', ylabel='Number of Loans', sort=True)

Create Feature Column for Underwriting TRUE

In [ ]:
def map_underwriting_true(*args):
    columns = [col for col in args]
    for column in columns:
        if column == True:
            return 1
        elif column == False:
            return 0
        
df['underwriting_true'] = list(map(map_underwriting_true, df.underwritingindicator))
df.drop(['underwritingindicator'], axis=1, inplace=True)
pd.value_counts(df.underwriting_true)

# Subvented

In [ ]:
df.subvented.unique()

In [ ]:
df['subvented']=df.subvented.astype('category')

In [ ]:
draw(df.subvented, title='Subvented', ylabel='Number of Loans', sort=True)

In [ ]:
df.subvented.describe(include='all')

Create Feature Column for Subvented

In [ ]:
# create feature columns for subvented
def map_subvented(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.subvented).index.tolist()

for code in codes:
        df['subvented_{}'.format(code)] = list(map(map_subvented, df.subvented))
        print('Added Column for :   subvented_'+code)
print('New Dataframe shape : ', df.shape)
df.drop(['subvented'], axis=1, inplace=True)

# Vehicle New or Used code

In [ ]:
df.vehiclenewusedcode.unique()

In [ ]:
df['vehiclenewusedcode']=df.vehiclenewusedcode.astype('category')

In [ ]:
draw(df.vehiclenewusedcode, title='Vehicle New or Used code', ylabel='Number of Loans', sort=True)

# Create Feature Column for New or Used

In [ ]:
def map_vehiclenewusedcode_true(*args):
    columns = [col for col in args]
    for column in columns:
        if column == 1:
            return 1
        else:
            return 0
        
df['vehiclenew_true'] = list(map(map_vehiclenewusedcode_true, df.vehiclenewusedcode))
df.drop(['vehiclenewusedcode'], axis=1, inplace=True)
pd.value_counts(df.vehiclenew_true)

# Vehicle Type code

In [ ]:
df.vehicletypecode.unique()

In [ ]:
df['vehicletypecode']=df.vehicletypecode.astype('category')

In [ ]:
draw(df.vehicletypecode, title='Vehicle Type Code', ylabel='Number of Loans', sort=True)

In [ ]:
df.vehicletypecode.describe()

In [ ]:
df.drop(['vehicletypecode'],axis=1, inplace=True)
df.shape

# Obligor Credit Score Type

In [ ]:
df.obligorcreditscoretype.unique()

In [ ]:
df['obligorcreditscoretype']=df.obligorcreditscoretype.astype('category')

In [ ]:
draw(df.obligorcreditscoretype, title='Obligor Credit Score Type', ylabel='Number of Loans', sort=True)

In [ ]:
df.obligorcreditscoretype.describe()

In [ ]:
pd.value_counts(df.obligorcreditscoretype==0)

Create features for obligator credit score type

In [ ]:
def map_obligorcreditscoretype(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.obligorcreditscoretype).index.tolist()

for code in codes:
        df['credit_type_{}'.format(code)] = list(map(map_obligorcreditscoretype, df.obligorcreditscoretype))

In [ ]:
df.drop(['obligorcreditscoretype'], axis=1, inplace=True)
df.shape

# Obligor Credit Score

In [ ]:
#df.obligorcreditscore.isnull().values.any()

In [ ]:
#df.obligorcreditscore.value_counts(dropna=False).sort_values(ascending=False).head()

In [ ]:
#df.obligorcreditscore.value_counts('NaN').head()

In [ ]:
draw(df.obligorcreditscore, title='Obligor Credit Score', ylabel='Number of Loans', sort=True, n=10)

In [ ]:
df.obligorcreditscore.describe()

In [ ]:
#search for strings in the columns
q=df.obligorcreditscore.astype(str).str.contains(r'^([^0-9]*)$', na=False).sum()
print('Number of Strings in the Columns : ', q)
if q > 0:
    u=df[df.obligorcreditscore.astype(str).str.contains(r'^([^0-9]*)$', na=False)].obligorcreditscore.unique()
    print('Strings in data : ', u)
    df=df[~df.obligorcreditscore.astype(str).str.contains(r'^([^0-9]*)$', na=False)]
else:
    print('No strings in the Obligor Credit Score')
    


In [ ]:
#Number of blanks in the column
df.obligorcreditscore.astype(str).str.contains(r'^(\s+)$').sum()

In [ ]:
#pd.crosstab(df.reportingperiodendingdate, df.obligorcreditscore)

In [ ]:

#df=df[~df.obligorcreditscore.str.contains(r'^([^0-9]*)$', na=False)]

In [ ]:
df.isnull().sum()

In [ ]:
#df.obligorcreditscore.isnull().value_counts()

In [ ]:
#df[df.obligorcreditscore.isnull()]

In [ ]:
df=df.dropna(subset=['obligorcreditscore'])

In [ ]:
df.obligorcreditscore.value_counts(dropna=False).sort_values(ascending=False).head()

In [ ]:
df.obligorcreditscore.isnull().values.any()

In [ ]:
df.dtypes

# Obligor Income Verification Level Code

In [ ]:
df['obligorincomeverificationlevelcode']=df.obligorincomeverificationlevelcode.astype('category')

In [ ]:
draw(df.obligorincomeverificationlevelcode, title='Obligor Credit Score', ylabel='Number of Loans', sort=True)

In [ ]:
df.obligorincomeverificationlevelcode.describe()

In [ ]:
pd.value_counts(df.obligorincomeverificationlevelcode)

In [ ]:
u=pd.value_counts(df.obligorincomeverificationlevelcode==0)
if len(u)<2:
    print('No blank or zero obligor income verification level code')
else: 
    print('Number of loans with no obligor income verification level code',':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with obligor income verification level code')
    df=df[df.obligorincomeverificationlevelcode>0]
df.shape

# Create Feature Columns for Obligor Income Verification Code

In [ ]:
def map_obligorincomeverificationlevelcode(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.obligorincomeverificationlevelcode).index.tolist()

for code in codes:
        df['income_code{}'.format(code)] = list(map(map_obligorincomeverificationlevelcode, df.obligorincomeverificationlevelcode))


In [ ]:
df.drop(['obligorincomeverificationlevelcode'], axis=1, inplace=True)

# Obligor Employment Verification Code

In [ ]:
df['obligoremploymentverificationcode']=df.obligoremploymentverificationcode.astype('category')

In [ ]:
draw(df.obligoremploymentverificationcode, title='Obligor Employment Verification Code', ylabel='Number of Loans', sort=True)

In [ ]:
#pd.value_counts(df.obligoremploymentverificationcode==0)

In [ ]:
df.obligoremploymentverificationcode.describe()

In [ ]:
u=pd.value_counts(df.obligoremploymentverificationcode==0)
if len(u)<2:
    print('No blank or zero obligor employment verification level code')
else: 
    print('Number of loans with no obligor employment verification level code',':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with obligor employment verification level code')
    df=df[df.obligoremploymentverificationcode>0]
df.shape

# Create feature Columns for obligor employment verification

In [ ]:
def map_obligoremploymentverificationcode(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.obligoremploymentverificationcode).index.tolist()

for code in codes:
        df['empl_code{}'.format(code)] = list(map(map_obligoremploymentverificationcode, df.obligoremploymentverificationcode))


In [ ]:
df.drop(['obligoremploymentverificationcode'], axis=1, inplace=True)

# Co-obligator Indicator

In [ ]:
df['coobligorindicator']=df.coobligorindicator.astype('category')

In [ ]:
df.coobligorindicator.describe()

In [ ]:
draw(df.coobligorindicator, title='Co-Obligator Indicator', ylabel='Number of Loans', sort=True)

# Create features for Co-obligator Indicator True

In [ ]:
def map_coobligorindicator_true(*args):
    columns = [col for col in args]
    for column in columns:
        if column == 1:
            return 1
        else:
            return 0
        
df['coobligorindicator_true'] = list(map(map_coobligorindicator_true, df.coobligorindicator))
df.drop(['coobligorindicator'], axis=1, inplace=True)
pd.value_counts(df.coobligorindicator_true)

# Reporting Period Actual End Balance Amount

Mean reporting period actual end balance amount by the reporting period ending date

In [ ]:
df.groupby(df.reportingperiodendingdate).reportingperiodactualendbalanceamount.mean().plot(kind='bar')

In [ ]:
#draw(df.reportingperiodactualendbalanceamount, title='Reporting Period Actual End Balance Amount', ylabel='Number of Loans', sort=True)

In [ ]:
df.reportingperiodactualendbalanceamount.describe()

In [ ]:
u=pd.value_counts(df.reportingperiodactualendbalanceamount<=0)
if len(u)<2:
    print('No blank or zero or negative reporting period actual end balance amount')
else: 
    print('Number of loans with no reporting period actual end balance amount',':', u[1])
    print('Dropping ', u[1], 'rows and retaining loans with reporting period actual end balance amount')
    df=df[df.reportingperiodactualendbalanceamount>0]
df.shape

# Total Actual amount paid

In [ ]:
df.totalactualamountpaid.describe()

In [ ]:
u=pd.value_counts(df.totalactualamountpaid<=0)
if len(u)<2:
    print('No blank or zero or negative total actual amount paid')
else: 
    print('Number of loans with no or negative total actual amount paid',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with positive total actual amount paid')
    #df=df[df.totalactualamountpaid>0]
df.shape

# Actual interest collected amount

In [ ]:
df.actualinterestcollectedamount.describe()

In [ ]:
u=pd.value_counts(df.actualinterestcollectedamount<=0)
if len(u)<2:
    print('No blank or zero actual interest collected amount')
else: 
    print('Number of loans with no or negative actual interest collected amount',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with positive actual interest collected amount')
    #df=df[df.actualinterestcollectedamount>0]
df.shape

# Actual principal collected amount

In [ ]:
df.actualprincipalcollectedamount.describe()

In [ ]:
u=pd.value_counts(df.actualprincipalcollectedamount<=0)
if len(u)<2:
    print('No blank or zero actual principal collected amount')
else: 
    print('Number of loans with zero or negative actual principal collected amount',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with positive actual principal collected amount')
    #df=df[df.actualprincipalcollectedamount>0]
df.shape

# Interest paid through date

In [ ]:
pd.value_counts(df.interestpaidthroughdate).head(n=10)

In [ ]:
df.interestpaidthroughdate.describe()

In [ ]:
u=pd.value_counts(df.interestpaidthroughdate=='1900-01-01')
if len(u)<2:
    print('No blank or zero interest paid throughdate as 1900-01-01')
else: 
    print('Number of loans with zero or 1900-01-01 as  interest paid throughdate',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with correct interest paid throughdate')
    #df=df[df.interestpaidthroughdate>'1900-01-01']
df.shape

In [ ]:
df.drop(['interestpaidthroughdate'], axis=1, inplace=True)
df.shape

# Days from origination to first payment date

In [ ]:
df['daystofirstpayment'] = pd.to_datetime(df['originalfirstpaymentdate'])-pd.to_datetime(df['originationdate'])
#draw a chart for visual look and analysis
draw(df.daystofirstpayment, title='Days to First Payment', ylabel='Number of Loans', sort = True)


In [ ]:
df.daystofirstpayment.describe()

In [ ]:
#remove days string for machine learning
df['daystofirstpayment']=df.daystofirstpayment.map(lambda x: np.nan if pd.isnull(x) else x.days)
#display last five rows
df.daystofirstpayment.tail()

In [ ]:
u=pd.value_counts(df.daystofirstpayment==0)
if len(u)<2:
    print('No blank or zero days to first payment')
else: 
    print('Number of loans with zero days to first payment',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with positive total actual amount paid')
    #df=df[df.daystofirstpayment>0]
df.shape

In [ ]:
df.drop(['originalfirstpaymentdate'], axis=1, inplace=True)
df.shape

# Grace Period

In [ ]:
draw(df.graceperiodnumber, title='Grace period Number', ylabel='Number of Loans', sort = True)

In [ ]:
df.graceperiodnumber.describe()

In [ ]:
u=pd.value_counts(df.graceperiodnumber==0)
if len(u)<2:
    print('No blank or zero grace period number')
else: 
    print('Number of loans with zero grace period number',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with positive grace period number')
    #df=df[df.graceperiodnumber>0]
df.shape

# Asset added indicator

In [ ]:
df['assetaddedindicator']=df.assetaddedindicator.astype('category')

In [ ]:
draw(df.assetaddedindicator, title='Asset Added Indicator', ylabel='Number of Loans', sort = True)

In [ ]:
df.assetaddedindicator.describe()

In [ ]:
u=pd.value_counts(df.assetaddedindicator!=False)
if len(u)<2:
    print('No blank or zero Asset Added Indicator')
else: 
    print('Number of loans with zero or blank Asset Added Indicator',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with positive asset added indicator')
    #df=df[df.assetaddedindicator>0]
df.shape

In [ ]:
df.drop(['assetaddedindicator'], axis=1, inplace=True)
df.shape

# Reporting Period Modification Indicator

In [ ]:
df['reportingperiodmodificationindicator']=df.reportingperiodmodificationindicator.astype('category')

In [ ]:
draw(df.reportingperiodmodificationindicator, title='Reporting Period Modification Error', ylabel='Number of Loans', sort = True)

In [ ]:
df.reportingperiodmodificationindicator.describe()

# Create Features for Reporting Period Modification Indicator True

In [ ]:
def map_reportingperiodmodificationindicator_true(*args):
    columns = [col for col in args]
    for column in columns:
        if column == 1:
            return 1
        else:
            return 0
        
df['reportingperiodmodificationindicator_true'] = list(map(map_reportingperiodmodificationindicator_true, df.reportingperiodmodificationindicator))
df.drop(['reportingperiodmodificationindicator'], axis=1, inplace=True)
pd.value_counts(df.reportingperiodmodificationindicator_true)

In [ ]:
df.shape

# Servicing Advance Method Code

In [ ]:
df['servicingadvancemethodcode']=df.servicingadvancemethodcode.astype('category')

In [ ]:
draw(df.servicingadvancemethodcode, title='Serving Advance Method Code', ylabel='Number of Loans', sort = True)

In [ ]:
df.servicingadvancemethodcode.describe()

# Create Feature Columns for Servicing Advance Method Codes

In [ ]:
def map_servicingadvancemethodcode(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.servicingadvancemethodcode).index.tolist()

for code in codes:
        df['servicingadvancemethod_code{}'.format(code)] = list(map(map_servicingadvancemethodcode, df.servicingadvancemethodcode))


In [ ]:
df.drop(['servicingadvancemethodcode'], axis=1, inplace=True)
df.shape

# Other Principal Adjustment Amount

In [ ]:
df.otherprincipaladjustmentamount.describe()

In [ ]:
u=pd.value_counts(df.otherprincipaladjustmentamount!=True)
if len(u)<2:
    print('No blank other principal adjustment amount')
else: 
    print('Number of loans with zero other principal adjustment amount',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with other principal adjustment amount')
    #df=df[df.otherprincipaladjustmentamount>0]
df.shape

# Zero Balance Effective Date

In [ ]:
df.zerobalanceeffectivedate.describe()

# Creating new feature column for zero balance effective date

In [ ]:
for i in pd.value_counts(df.zerobalanceeffectivedate).keys().tolist():
    if i>cutOffForOriginationDate:
        df['zerobalanceeffectivedate_calc'] = pd.to_datetime(df['zerobalanceeffectivedate'])-pd.to_datetime(df['originationdate'])
df['zerobalanceeffectivedate_calc']=df.zerobalanceeffectivedate_calc.map(lambda x: np.nan if pd.isnull(x) else x.days)
#df.zerobalanceeffectivedate_calc.replace(np.nan, 0, inplace = True)

In [ ]:
df.zerobalanceeffectivedate_calc.describe()

In [ ]:
# fill NaN values with zeros
df.zerobalanceeffectivedate_calc.replace(np.nan, 0, inplace = True)

In [ ]:
df.drop(['zerobalanceeffectivedate'], axis=1, inplace=True)
df.shape

# Zero Balance Code

In [ ]:
df['zerobalancecode']=df.zerobalancecode.astype('category')

In [ ]:
draw(df.zerobalancecode, title='Zero Balance Code', ylabel='Number of Loans', sort = True)

# Create feature column for zero balance code

In [ ]:
def map_zerobalancecode(*args):
    columns = [col for col in args]
    for column in columns:
        if column == code:
            return 1
        else:
            return 0
codes = pd.value_counts(df.zerobalancecode).index.tolist()

for code in codes:
        df['zerobalance_code{}'.format(code)] = list(map(map_zerobalancecode, df.zerobalancecode))


In [ ]:
df.shape

In [ ]:
df.drop(['zerobalancecode'], axis=1, inplace=True)
df.shape

In [ ]:
# df.set_index('originationdate')
# df.sort_values(by='originationdate', ascending=True)

# Add S&P Experian default index and historical data

In [ ]:
df=df.set_index('reportingperiodendingdate')
#df.head()
#df.to_csv('c:users/ugy1/abs/datasets/df.csv')

In [ ]:
df1=pd.read_csv(path+'datasets/s&pExperianAutoDefaultIndex.csv', index_col=0)
df=df.join(df1)
df2=pd.read_csv(path+'datasets/experianHistDefaultRates.csv', index_col=0)
df=df.join(df2)

In [ ]:
df.head()

In [ ]:
#df=df.drop(['reportingperiodbeginningdate','originationdate"], axis=1)
df.drop(['cik', 'assetnumber', 'reportingperiodbeginningdate','originationdate'], axis=1, inplace=True)
df.shape
        

# Clean data

In [ ]:
# change columns to lower case and remove prefix to enable pandas to work
#column_list=[x.lower() for x in (column_list)]
#df.columns=df.columns.str.replace('abs_loan.','')
# convert objects to numeric
#df=df.convert_objects(convert_numeric=True)

# Check for null

In [ ]:
df.isnull().values.any()

# Loan Statistics

In [ ]:
df.describe()

# Current Vs Other Buckets

In [ ]:
LABELS = ('Current', 'Not Current')
count_classes = pd.value_counts(df.currentdelinquencystatus>0, sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.xticks(range(2), LABELS)
plt.title('Deliquency Class')
plt.ylabel("Number of Loans")
plt.show()
print('Number of loans Current :', count_classes[0],'   ', 'Number of loans Not Current :', count_classes[1],)

In [ ]:
#pd.value_counts(df.currentdelinquencystatus)

In [ ]:
df.currentdelinquencystatus.describe()

In [ ]:
u=pd.value_counts(df.currentdelinquencystatus==''.index) 
if len(u)<2:
    print('No blank current delinquency status')
else: 
    print('Number of loans with blank current delinquency status',':', u[1])
    #print('Dropping ', u[1], 'rows and retaining loans with correct interest paid throughdate')
    #df=df[df.interestpaidthroughdate>'1900-01-01']
df.shape

# Label Creation

In [ ]:
def map_label(*args):
    columns = [col for col in args]
    for column in columns:
        if column > 0:
            return 1
        elif column == 0:
            return 0
        
df['label'] = list(map(map_label, df.currentdelinquencystatus ))

LABELS = ('Current', 'Default')
count_classes = pd.value_counts(df.label, sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.xticks(range(2), LABELS)
plt.title('Revised Deliquency Class')
plt.ylabel("Number of Loans")
plt.show()
print('Number of loans Current :', count_classes[0],'   ', 'Number of loans Defaulted :', count_classes[1],)

# Loan Balance Amount at the End of Reporting Period

In [ ]:
current_df = df[df.label==0]
not_current_df = df[df.label==1]
bins = np.linspace(0, 100000, 100)
plt.hist(current_df.reportingperiodactualendbalanceamount, bins, alpha=0.6, 
         #density=True, 
         label='CURRENT')
plt.hist(not_current_df.reportingperiodactualendbalanceamount, bins, alpha=1, 
         #density=True, 
         label='NOT CURRENT')
plt.legend(loc='upper right')
plt.title("Loan Balance Amount at the end of reporting Period")
plt.xlabel('Loan Balance Amount')
plt.ylabel('Number of of Loans')
plt.show()

# Manual oversampling of default data 

In [ ]:
#def manual_oversampling(df):
default=df[df.label==1]
print('default: ',default.shape)
b=int((df.shape[0]-default.shape[0])/(default.shape[0]))
print('copy default data ',b, ' times')
b=int(b*manualOversamplingFactor)
print(b)
for i in range(b):
    df=df.append(default, ignore_index=True)
df=df.sample(frac=1).reset_index(drop=True)
draw(df.label, title='Manual Oversampling -Current Vs Non-Current', ylabel='Number of Loans', sort=True)
print(df.shape)
print(pd.value_counts(df.label==1))
#return df
   

In [ ]:
#manual_oversampling(df)
print(pd.value_counts(df.label==1))
df.shape

In [ ]:
df.dtypes

# Oversampling with SMOTE

In [ ]:
# from imblearn.over_sampling import SMOTE, ADASYN
# x_train, y_train = SMOTE().fit_resample(x_train, y_train)

In [ ]:
# #Apply SMOTE to create synthetic observations from minority class
# #Source:https://github.com/scikit-learn-contrib/imbalanced-learn
# from imblearn.over_sampling import SMOTE #Over sampling
# sm = SMOTE(ratio='auto',kind='regular')
# X_sampled,y_sampled = sm.fit_sample(X,y.values.ravel())

# #Percentage of fraudlent records in original data
# Source_data_no_fraud_count = len(data[data.Class==0])
# Source_data_fraud_count = len(data[data.Class==1])
# print('Percentage of fraud counts in original dataset:{}%'.format((Source_data_fraud_count*100)/(Source_data_no_fraud_count+Source_data_fraud_count)))

# #Percentage of fraudlent records in sampled data
# Sampled_data_no_fraud_count = len(y_sampled[y_sampled==0])
# Sampled_data_fraud_count = len(y_sampled[y_sampled==1])
# print('Percentage of fraud counts in the new data:{}%'.format((Sampled_data_fraud_count*100)/(Sampled_data_no_fraud_count+Sampled_data_fraud_count)))

# Naive Random Oversampling

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=0)
# X_resampled, y_resampled = ros.fit_resample(X, y)
# from collections import Counter
# print(sorted(Counter(y_resampled).items()))

#  Clean up for Machine Learning

In [ ]:
df.shape

In [ ]:
#df=df.replace([np.inf, -np.inf], np.nan)
#df=df.fillna(0, inplace=True)
#df=df.dropna(inplace=True)
#df.head()
df.head()

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().any().sum()

In [ ]:
pd.value_counts(df.label, dropna=False)

In [ ]:
df.label.fillna(0, inplace=True)

In [ ]:
pd.value_counts(df.label, dropna=False)

In [ ]:
pd.value_counts(df.paymenttoincomepercentage, dropna=False).head()

In [ ]:
df.paymenttoincomepercentage.dropna(inplace=True)

In [ ]:
pd.value_counts(df.paymenttoincomepercentage, dropna=False).head()

In [ ]:
df['servicingflatfeeamount']=df.servicingflatfeeamount.astype('float')
pd.value_counts(df.servicingflatfeeamount, dropna=False) 

In [ ]:
df.servicingflatfeeamount.fillna(0,inplace=True)

In [ ]:
pd.value_counts(df.reportingperiodbeginningloanbalanceamount, dropna=False).head()

In [ ]:
df.reportingperiodbeginningloanbalanceamount.fillna(0, inplace=True)

In [ ]:
pd.value_counts(df.reportingperiodbeginningloanbalanceamount, dropna=False).head()

In [ ]:
pd.value_counts(df.otherservicerfeeretainedbyservicer, dropna=False).head()

In [ ]:
df.otherservicerfeeretainedbyservicer.fillna(0, inplace=True)

In [ ]:
pd.value_counts(df.otherservicerfeeretainedbyservicer, dropna=False).head()

In [ ]:
pd.value_counts(df.serviceradvancedamount, dropna=False).head()

In [ ]:
df.serviceradvancedamount.fillna(0, inplace=True)

In [ ]:
pd.value_counts(df.serviceradvancedamount, dropna=False).head()

In [ ]:
df.isnull().any()

In [ ]:
pd.value_counts(df.paymenttoincomepercentage, dropna=False).head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
#df.isnull().any().sum()

In [ ]:
df.shape 

# Preparing data for Machine Learning

In [ ]:
# prepare label for scikit-learn
Y=df.label.values
Y.shape

In [ ]:
# reshape labels for machine learning
#Y=Y.reshape(Y.shape[0],1)
Y.shape

In [ ]:
# prepare input data for scikit-learn
input=df.values
input.shape

In [ ]:
# calculate train/test split
len_train = int(len(input)*train_split)
print(len_train)

In [ ]:
# apply train/test split to labels
y_train = Y[0:len_train]
y_test = Y[len_train:]

In [ ]:
#reshape input data for machine learning
#input=input.reshape(input.shape[0], input.shape[1], 1)
input.shape

In [ ]:
# apply train/test split to input data
x_train = input[0:len_train]
x_test = input[len_train:]
x_train.shape

# Scikit-Learn Scaling

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import minmax_scale
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import QuantileTransformer
# from sklearn.preprocessing import PowerTransformer

In [ ]:
# x_scaler=MinMaxScaler()
# x_train = x_scaler.fit_transform(x_train)
# x_test = x_scaler.fit_transform(x_test)

# Keras Normalize input data

In [ ]:
x_train_keras_scaled = keras.utils.normalize(x_train, axis=-1, order=2)
x_test_keras_scaled = keras.utils.normalize(x_test, axis=-1, order=2)
x_train_keras_scaled.shape

# Linear SVC

In [ ]:
from sklearn import svm, neighbors
lsvc=svm.LinearSVC()
clf_lsvc=lsvc.fit(x_train, y_train)
confidence_lsvc=clf_lsvc.score(x_test, y_test)

# K-Nearest Neighbors

In [ ]:
knn=neighbors.KNeighborsClassifier(n_neighbors=10, n_jobs=-1)
clf_knn=knn.fit(x_train, y_train)
confidence_knn=clf_knn.score(x_test, y_test)

# Random Forest Tree

In [ ]:
rfor=RandomForestClassifier()
clf_rfor = rfor.fit(x_train, y_train)
confidence_rfor=clf_rfor.score(x_test, y_test)

# Logistic Regression

In [ ]:
lr = LogisticRegression(solver='lbfgs')
clf_lr = lr.fit(x_train, y_train)
confidence_lr=clf_lr.score(x_test, y_test)

# Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
clf_gnb = gnb.fit(x_train, y_train)
confidence_gnb=clf_gnb.score(x_test, y_test)

# Gated Recurring Unit

In [ ]:

# model = keras.Sequential()
# model.add(keras.layers.GRU(128, activation='relu',  
#                            kernel_regularizer=regularizers.l2(0.01), 
#                            input_shape=(x_train_keras_scaled.shape[1:]), 
#                            return_sequences=True))
# model.add(keras.layers.Dropout(0.3))

# model.add(keras.layers.GRU(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(keras.layers.Dropout(0.2))

# model.add(keras.layers.Dense(32, activation='relu',  kernel_regularizer=regularizers.l2(0.01)))
# model.add(keras.layers.Dropout(0.3))

# model.add(keras.layers.Dense(2, activation='softmax'))

# optimizer = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-6)
# model.summary()

# Model Configuration

In [ ]:
#save the best model
#keras.callbacks.Callback()
# checkPoint = ModelCheckpoint(filepath = path+'model/'+name+'.h5',
#                             save_best_only = True,
#                             verbose=0)
# tensorboard = TensorBoard(log_dir=path+'logs/{}'.format(name),
#                           histogram_freq=0,
#                           batch_size=32,
#                           write_graph=True,
#                           write_images=False)
# history = keras.callbacks.History()

# Train the Model

In [ ]:
# model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test_keras_scaled, y_test), batch_size=254, 
#         # callbacks = [
#    # baseLogger,
#     #history,
#     #tensorboard
#     #learningRateScheduler,
#     #reduceLROnPlateau
#      #    ], 
#           shuffle=False
# )


# Save the Model

In [ ]:
# model.save(path+'model/'+name+'.h5')

In [ ]:
# test_loss, test_acc = model.evaluate(x_test_keras_scaled, y_test)
# print('Test Loss : ',test_loss,' ','Test Accuracy : ', test_acc)

In [ ]:
# y_pred = model.predict_classes(x_test)

In [ ]:
#print(y_pred)

In [ ]:
#len(y_pred)

In [ ]:
# np.count_nonzero(y_pred)

In [ ]:
# np.count_nonzero(y_test)

# Load Saved Model

In [ ]:
#saved_model = load_model(path+'model/ABS_Keras-GRU254GRU128D32D2-adam1540234475.h5')

In [ ]:
# test_loss, test_acc = saved_model.evaluate(x_test_keras_scaled, y_test)
# print('Test Loss : ',test_loss,' ','Test Accuracy : ', test_acc)

# Predict Classes

In [ ]:
x_pred = x_test

In [ ]:
#prediction_gru = model.predict_classes(x_pred)
prediction_lsvc = clf_lsvc.predict(x_pred)
prediction_knn = clf_knn.predict(x_pred)
prediction_rfor = clf_rfor.predict(x_pred)
prediction_lr = clf_lr.predict(x_pred)
prediction_gnb = clf_gnb.predict(x_pred)

In [ ]:
predictions = [
  # prediction_gru,
    prediction_lsvc,
    prediction_knn,
    prediction_rfor,
    prediction_lr,
    prediction_gnb 
]

# Confusion Matrix

In [ ]:
class_names = ['Current', 'Non-Current']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

for prediction in predictions:
        print('ROC_AUC_SCORE ; ', roc_auc_score(y_test, prediction))
        # Compute confusion matrix
        cnf_matrix = confusion_matrix(y_test, prediction)
        np.set_printoptions(precision=2)

        # Plot non-normalized confusion matrix
        plt.figure()
        plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title= 'Confusion matrix, without normalization')

        # Plot normalized confusion matrix
        plt.figure()
        plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=False,
                          title='Normalized confusion matrix')

        plt.show()

In [ ]:
# fpr, tpr, thresholds = metrics.roc_curve(y_test, prediction, pos_label=2)
# metrics.auc(fpr, tpr)

In [ ]:
#1. find any string which has no numbers 

#2. find only numbers r'^([0-9]*)$'

#3. find only blanks(empty strings) r'^(\s+)$'

In [ ]:
#at least one Nan in a row
# df1=df[df.isnull().any(axis=1)]
# df.isna()
# df.notna()
# df.loc[:,df.isna().any]

In [ ]:
# #replace NaN with zero
# df.fillna(0, inplace=True)